# Gradient Boost Regressor - Model Evaluation

- start with less features

In [5]:
# standard modules
import sys
import os

# data and vizualisation models
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

# machine learning
import joblib
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_validate, learning_curve, train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Get the current working directory
current_dir = os.getcwd()

# add 'main' to the path
main_dir = os.path.dirname(current_dir)
sys.path.append(main_dir)

# data preprocessing and scaling
from main.feature_engin.feature_import import combine_dataframes


Downloading the preprocessed and scaled (only traget not scaled) dataset from Google Cloud Bucket

In [6]:
# fetching data
raw_data = combine_dataframes()
# copying
data = raw_data.copy()
data.info()


Loaded coal_price.csv successfully.
Loaded ttf_price.csv successfully.
Loaded oil_price.csv successfully.
Loaded germany_electricity_generation_2018-2023.csv successfully.
Loaded holidays.csv successfully.
Loaded PMI_germany.csv successfully.
Loaded weather_north_hourly.csv successfully.
Loaded weather_south_hourly.csv successfully.
Loaded weather_brocken_hourly.csv successfully.
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 199389 entries, 2018-01-02 00:00:00+00:00 to 2023-11-22 00:00:00+00:00
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   coal_adj_close          199389 non-null  float64
 1   ttf_adj_close           199389 non-null  float64
 2   ttf_volume              199389 non-null  float64
 3   oil_adj_close           199389 non-null  float64
 4   oil_volume              199389 non-null  float64
 5   fractional_hour         199389 non-null  float64
 6   day_of_week            

In [7]:
# sortby the datetime index
data = data.sort_index()

# Identify the Split Point
split_date = data.index.max() - pd.Timedelta(weeks=2)

data_mod = data[data.index <= split_date]
last_two_weeks_data = data[data.index > split_date]

# Check the number of entries in the split datasets
print("Data up to the last two weeks:", data_mod.shape[0])
print("Last two weeks data:", last_two_weeks_data.shape[0])


Data up to the last two weeks: 198045
Last two weeks data: 1344


In [8]:
last_two_weeks_data.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1344 entries, 2023-11-08 00:15:00+00:00 to 2023-11-22 00:00:00+00:00
Data columns (total 45 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   coal_adj_close          1344 non-null   float64
 1   ttf_adj_close           1344 non-null   float64
 2   ttf_volume              1344 non-null   float64
 3   oil_adj_close           1344 non-null   float64
 4   oil_volume              1344 non-null   float64
 5   fractional_hour         1344 non-null   float64
 6   day_of_week             1344 non-null   float64
 7   week_of_year            1344 non-null   float64
 8   month                   1344 non-null   float64
 9   year                    1344 non-null   float64
 10  hydro_storage_in        1344 non-null   float64
 11  cross_border            1344 non-null   float64
 12  nuclear                 1344 non-null   float64
 13  hydro                   1344 non-null   float

In [9]:
# Separating the features and the target variable from the original dataset
X = data_mod.drop(columns=['day_ahead_price'])
y = data_mod['day_ahead_price']

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:

# Features for MinMaxScaler as defined by the user
features_min_max_scaler = ['hydro_storage_in', 'cross_border', 'nuclear', 'hydro', 'biomass', 'lignite', 'hard_coal', 'oil', 'nat_gas', 'geothermal', 'hydro_reservoir', 'hydro_storage_out', 'others', 'wind_offshore', 'wind_onshore', 'solar', 'load', 'residual_load']

# Create the MinMaxScaler transformer
min_max_scaler_transformer = Pipeline(steps=[
    ('min_max_scaler', MinMaxScaler())
])

# Create the combined transformer with only the MinMaxScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('minmax_scaler', min_max_scaler_transformer, features_min_max_scaler)
    ],
    remainder='drop'  # This will pass drop other columns without transformation
)

# Apply the scaling transformation to the original datasets
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)


# Gradient Boost Regressor
## Instantiate the Gradient Boost Regressor Model

- Scoring method is R^2 to penalize high errors with huber loss function
- best model after extensive gridsearch
- increased max dept to improve model fitting
- huber loss to decrease error for extreme values (option: tweaking alpha value)

In [14]:
# Best parameters from your previous grid search, with 'huber' loss
best_params_huber = {
    'n_estimators': 300,
    'learning_rate': 0.2,
    'max_depth': 12,
    'loss': 'huber',
    'min_samples_split': 10
}

# Create the model with 'huber' loss
best_gbr = GradientBoostingRegressor(**best_params_huber, random_state=42)

# Training the model
best_gbr.fit(X_train_scaled, y_train)

# Predicting the target values for the test set
y_pred = best_gbr.predict(X_test_scaled)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Displaying the evaluation metrics
print("Mean Absolute Error with 'huber' loss:", mae)
print("Mean Squared Error with 'huber' loss:", mse)
print("R² Score with 'huber' loss:", r2)

Mean Absolute Error with 'huber' loss: 4.818644681339762
Mean Squared Error with 'huber' loss: 109.69477058456154
R² Score with 'huber' loss: 0.9890277964670594


In [15]:
# Best parameters from your previous grid search, with 'huber' loss
best_params_huber = {
    'n_estimators': 300,
    'learning_rate': 0.1,
    'max_depth': 8,
    'loss': 'huber',
    'min_samples_split': 10
}

# Create the model with 'huber' loss
best_gbr = GradientBoostingRegressor(**best_params_huber, random_state=42)

# Training the model
best_gbr.fit(X_train_scaled, y_train)

# Predicting the target values for the test set
y_pred = best_gbr.predict(X_test_scaled)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Displaying the evaluation metrics
print("Mean Absolute Error with 'huber' loss:", mae)
print("Mean Squared Error with 'huber' loss:", mse)
print("R² Score with 'huber' loss:", r2)

Mean Absolute Error with 'huber' loss: 7.129857229575267
Mean Squared Error with 'huber' loss: 208.01857590700888
R² Score with 'huber' loss: 0.9791929720868081


In [16]:
# Best parameters from your previous grid search, with 'huber' loss
best_params_huber = {
    'n_estimators': 400,
    'learning_rate': 0.2,
    'max_depth': 16,
    'loss': 'huber',
    'min_samples_split': 12
}

# Create the model with 'huber' loss
best_gbr = GradientBoostingRegressor(**best_params_huber, random_state=42)

# Training the model
best_gbr.fit(X_train_scaled, y_train)

# Predicting the target values for the test set
y_pred = best_gbr.predict(X_test_scaled)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Displaying the evaluation metrics
print("Mean Absolute Error with 'huber' loss:", mae)
print("Mean Squared Error with 'huber' loss:", mse)
print("R² Score with 'huber' loss:", r2)

Mean Absolute Error with 'huber' loss: 4.385063844746182
Mean Squared Error with 'huber' loss: 100.87372804935205
R² Score with 'huber' loss: 0.989910119968474


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Defining the parameter grid around the best parameters
param_grid = {
    'n_estimators': [350, 400, 450],  # values around 400
    'learning_rate': [0.1, 0.2, 0.3],  # values around 0.2
    'max_depth': [14, 16, 18],         # values around 16
    'min_samples_split': [10, 12, 14], # values around 12
    'loss': ['huber']                  # keeping the loss constant as 'huber'
}

# Create the GridSearchCV object
grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # You might use 'r2' or other metrics as well
    cv=5,  # Cross-validation strategy
    n_jobs=-1,  # Use all available cores
    verbose=2
)

# Perform the grid search on your training data
grid_search.fit(X_train_scaled, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score (negative MSE):", grid_search.best_score_)

# You can then evaluate the best estimator on your test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R² Score:", r2)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=350; total time=31.7min
[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=350; total time=33.6min
[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=350; total time=33.8min
[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=350; total time=34.1min
[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=400; total time=37.9min
[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=350; total time=37.9min
[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=400; total time=38.8min
[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=400; total time=40.4min
[CV] END learning_rate=0.1, loss=huber, max_depth=14, min_samples_split=10, n_estimators=400; to